In [1]:
import pandas as pd
import json
import numpy as np
import tiktoken

In [2]:
debates_df = pd.read_json("../data/filtered_data/debates_titles.json")
rounds_df = pd.read_json("../data/processed_data/rounds_df.json")
users_df = pd.read_json("../data/processed_data/users_df.json")
votes_df = pd.read_json("../data/filtered_data/votes_filtered_df.json")
propositions_df = pd.read_json("../data/raw_data/propositions.json")

/tmp/ipykernel_3716142/2766757945.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  users_df = pd.read_json("../data/processed_data/users_df.json")
/tmp/ipykernel_3716142/2766757945.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  users_df = pd.read_json("../data/processed_data/users_df.json")
/tmp/ipykernel_3716142/2766757945.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as dateti

In [3]:
# what we already have of crowdsourcing data
crowd1_df = pd.read_csv("../data/raw_data/crowdsourcing1.csv")
crowd2_df = pd.read_csv("../data/raw_data/crowdsourcing2.csv")
crowd3_df = pd.read_csv("../data/raw_data/crowdsourcing3.csv")
crowd = pd.concat([crowd1_df, crowd2_df, crowd3_df]).reset_index(drop=True)

crowd = crowd.rename(
    columns={"Input.debate_id": "debate_id", "Input.voter_id": "voter_id"}
)

crowd_debates = list(crowd.debate_id.unique())

In [4]:
abortion_debates = propositions_df[
    propositions_df.proposition.str.lower().str.contains("abortion")
].debate_id.unique()

gay_marriage_debates = propositions_df[
    (
        propositions_df.proposition.str.lower().str.contains("same sex")
        | propositions_df.proposition.str.lower().str.contains("gay")
        | propositions_df.proposition.str.lower().str.contains("same-sex")
    )
    & (propositions_df.proposition.str.lower().str.contains("marriage"))
].debate_id.unique()

capital_punishment_debates = propositions_df[
    (
        propositions_df.proposition.str.lower().str.contains("death penalty")
        | propositions_df.proposition.str.lower().str.contains("capital punishment")
    )
].debate_id.unique()

In [5]:
demographic_columns = [
    "birthday",
    "education",
    "ethnicity",
    "gender",
    "income",
    "party",
    "political_ideology",
    "religious_ideology",
]

In [6]:
def get_debate(rounds_df, debate_id: int) -> str:
    """Return the debate with id `debate_id`.

    The format of the debate is as follows:
    {Round 0: {
        Pro: pro argument,
        Con: con argument,
        },
     Round 1: {
        Pro: pro argument,
        Con: con argument,
        },
     etc.
    }
    """
    rounds = {}
    debate_df = rounds_df[rounds_df.debate_id == debate_id]

    for _, row in debate_df.iterrows():
        round_key = f"Round {row['round']}"
        if round_key in rounds.keys():
            old_dict = rounds[round_key]
            old_dict[row.side] = row.text
            rounds[round_key] = old_dict
        else:
            rounds[round_key] = {row.side: row.text}

    debate = str(json.dumps(rounds))
    return debate


def get_user_info(
    users_df: pd.DataFrame, voter_id: str, demographic_columns: list[str]
) -> str:
    """Return a string containing all the demographic information of user `voter_id`
    in the following format: Label: Value, Label: Value
    """
    voter_info = users_df.loc[voter_id][demographic_columns]
    voter_info = voter_info[~voter_info.isna()]
    voter_info = [
        col.replace("_", " ").title() + ": " + voter_info[col]
        for col in voter_info.index
    ]

    return "\n".join(voter_info)

In [7]:
debates_df = debates_df[
    debates_df.debate_id.isin(abortion_debates)
    | debates_df.debate_id.isin(gay_marriage_debates)
    | (debates_df.debate_id.isin(capital_punishment_debates))
]

In [8]:
debates_df["debate"] = debates_df.apply(
    lambda x: get_debate(rounds_df, x.debate_id), axis=1
)
# debates_df["word_count"] = debates_df.apply(
#     lambda x: len(tiktoken.encoding_for_model("gpt-3.5-turbo").encode(x.debate))
#     * (750 / 1000),
#     axis=1,
# )
# debates_df = debates_df[debates_df["word_count"] <= 1500]

In [9]:
df = debates_df.merge(votes_df)
df["user_info"] = df.apply(
    lambda x: get_user_info(users_df, x.voter_id, demographic_columns), axis=1
)
df = df[["debate_id", "voter_id", "proposition", "debate", "user_info"]]

In [10]:
df = df[~df.debate_id.isin(crowd_debates)]

In [11]:
df = df.drop_duplicates(subset=["debate_id", "voter_id"])

In [17]:
df.to_csv("../data/processed_data/mturk.csv")